In [1]:
import csv
import pymongo
from pymongo import MongoClient
import datetime as dt
import calendar
import pandas as pd
import numpy as np
import time
#These are all the imports to be used

In [2]:
c_v1 = MongoClient("mongodb://3.20.189.28:2036/momoKashBd?readPreference=secondary&directConnection=true")
db_v1 = c_v1["momoKashBd"]
#These are my collections
Clients = db_v1["Client"]
#Loans = db_v1["Loan"]
#Refunds = db_v1["Refund"]
print("MongoDB connection established with success")

MongoDB connection established with success


In [3]:
# Cell 3 - Batch Loader Function
def get_clients_by_batch(collection, projection, batch_size=10000):
    all_data = []
    skip = 0

    # Filter: subscriptionDate between Sept 1, 2022 and Sept 1, 2025
    date_filter = {
    "subscriptionDate": {
        "$gte": "01/09/2022",
        "$lt": "01/09/2025"
    }
}


    while True:
        cursor = collection.find(date_filter, projection).skip(skip).limit(batch_size)
        batch = list(cursor)
        if not batch:
            break

        all_data.extend(batch)
        skip += batch_size
        if len(all_data) % 10000 == 0:
            print(f"Traité {len(all_data)} documents...")

    return pd.DataFrame(all_data)


In [7]:
# Cell 4 - Fetch Clients in Batches
start = time.time()
projection ={"subscriptionDate": 1,"unsubscriptionDate":1,
    "clientId": 1,
    "loanId": 1,       
    "isBlacklisted": 1,
    "isBlocked": 1,
    "isEligible": 1,
    "isActive": 1,
    "isOwing": 1,
    "debt": 1,
    "dailyLoanCount": 1,
    "clientMaxAmount": 1,
    "dueAmount": 1,
    "totalRefunded": 1,
    "penaltyDebt":1
}

clients_df = get_clients_by_batch(Clients,projection)

end = time.time()
print("Fetched clients:", clients_df.shape)
print(f"Fetching time: {round((end-start)/60, 2)} minutes")
print(clients_df.head(3))


Fetched clients: (3642, 16)
Fetching time: 0.7 minutes
            _id  isBlacklisted  isBlocked  isEligible  isActive  isOwing  \
0  237653610519          False      False        True      True    False   
1  237679242287          False      False       False      True    False   
2  237675118864          False      False       False      True    False   

   dailyLoanCount    debt  clientMaxAmount  dueAmount  totalRefunded  \
0               0  1233.0          12179.0    10750.0        10324.0   
1               0  5647.0              0.0     6900.0         1253.0   
2               0  5912.0            468.0     7525.0         1613.0   

    clientId subscriptionDate  \
0  653610519       01/09/2022   
1  679242287       01/09/2022   
2  675118864       01/09/2022   

                                              loanId  unsubscriptionDate  \
0  237653610519:CAM07X01T202409070603360088M53610... 2025-09-06 23:00:00   
1  237679242287:CAM07X01T20220902113334MU:4641128524 2023-09-01 23

In [8]:
# Clean & Convert Dates, Fill Missing, Reset Index
date_cols = ["subscriptionDate", "unsubscriptionDate"]
for col in date_cols:
    clients_df[col] = pd.to_datetime(clients_df[col], dayfirst=True, errors="coerce")

num_cols = ["dailyLoanCount", "debt", "dueAmount", "totalRefunded", "clientMaxAmount", "penaltyDebt"]
clients_df[num_cols] = clients_df[num_cols].fillna(0)

clients_df.index = np.arange(clients_df.shape[0])

print("Sample after cleaning:")
display(clients_df.sample(5))

Sample after cleaning:


,_id,isBlacklisted,isBlocked,isEligible,isActive,isOwing,dailyLoanCount,debt,clientMaxAmount,dueAmount,totalRefunded,clientId,subscriptionDate,loanId,unsubscriptionDate,penaltyDebt
3154,237653164107,False,False,True,True,False,1,2866.0,2666.0,2866.0,0.0,653164107,2024-09-01,237653164107:CAM07X01T202511120904150236M53164...,2026-10-30 23:00:00.000,0.0
1372,237651980106,False,False,True,True,False,0,0.0,5081.0,0.0,0.0,651980106,2022-09-01,237651980106:CAM07X01T202307041850110030MP5198...,2023-09-01 23:00:00.000,0.0
2262,237651282008,False,False,True,True,False,0,0.0,10300.0,0.0,0.0,651282008,2024-09-01,237651282008:XRB011X01T202409010843290207M5128...,2025-08-31 23:00:00.000,0.0
1772,237675475280,False,False,True,True,False,0,0.0,636.0,0.0,0.0,675475280,2022-09-01,,NaT,0.0
3631,237652500857,False,False,True,True,False,0,0.0,500.0,0.0,0.0,652500857,2024-09-01,NaN,2029-09-01 22:35:17.504,0.0


In [10]:
# Cell 6 - Export in 2 halves for Excel
half = len(clients_df) // 2  # roughly half the rows

# First half
clients_df.iloc[:half].to_csv("clients_collection_part1.csv", index=False)
print(f"Exported clients_collection_part1.csv with {half} rows")

# Second half
clients_df.iloc[half:].to_csv("clients_collection_part2.csv", index=False)
print(f"Exported clients_collection_part1.csv with {len(clients_df) - half} rows")



Exported clients_collection_part1.csv with 1821 rows
Exported clients_collection_part1.csv with 1821 rows
